In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **LIB I USE**

bassic lib

In [ ]:
import os

from glob import glob
from tqdm import tqdm as print_progress
from datetime import datetime, timedelta, date
import datetime
import dateutil

import math
import numpy as np
import pandas as pd

import time
import warnings
import random

from joblib import Parallel, delayed
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns

from itertools import combinations
from joblib import Parallel, delayed
from collections import Counter
import scipy

featuretool

In [ ]:
import featuretools as ft
from featuretools.variable_types import *
from featuretools.utils.gen_utils import make_tqdm_iterator

from featuretools.primitives.base import *
from featuretools.utils.gen_utils import Library
from featuretools.primitives.standard.binary_transform import (
    AddNumeric, AddNumericScalar,
    SubtractNumeric, SubtractNumericScalar, ScalarSubtractNumericFeature,
    MultiplyNumeric, MultiplyNumericScalar,
    DivideNumeric, DivideNumericScalar, DivideByFeature,
    ModuloNumeric, ModuloNumericScalar, ModuloByFeature,
    GreaterThan, GreaterThanScalar, GreaterThanEqualTo, GreaterThanEqualToScalar,
    LessThan, LessThanScalar, LessThanEqualTo, LessThanEqualToScalar, 
    Equal, EqualScalar, NotEqual, NotEqualScalar
)

from featuretools.variable_types.variable import *
from featuretools.variable_types import Id, Numeric, Categorical, Datetime

**lib of ML**

In [ ]:
from sklearn.utils import class_weight
from sklearn.cluster import KMeans, MeanShift, DBSCAN
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.impute import SimpleImputer as sk_imputer

from sklearn.metrics import (
    classification_report, confusion_matrix, 
    silhouette_score, homogeneity_score, completeness_score, v_measure_score,
    auc, plot_roc_curve
  )

from sklearn.model_selection import (
    KFold, StratifiedKFold, cross_val_score, 
    GridSearchCV, 
    train_test_split
  )

from xgboost import XGBClassifier

from sklearn.base import BaseEstimator
import sklearn.linear_model as models
from sklearn.preprocessing import StandardScaler
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

# **all def and class for feature** 

**utils**

In [ ]:
def to_datetime(text: str):
    try:
        dt_format = str(parse_to_datetime(text))
        dt_object = datetime.strptime(dt_format,'%Y-%m-%d %H:%M:%S')
    except Exception:
        return None
    return dt_objec

def month_to_quarter(month: int) -> int:
    if 1 <= month <= 3:
        return 1
    elif 4 <= month <= 6:
        return 2
    elif 7 <= month <= 9:
        return 3
    elif 10 <= month <= 12:
        return 4
    else:
        raise ValueError(f'input must be between 1 and 12')

def load_csv(filepath: str) -> pd.DataFrame:
  if not os.path.isfile(filepath):
      raise FileNotFoundError(f"Cannot find {filepath}")
  return pd.read_csv(filepath)

LUCKY_NUMBER=4444
def find_best_k_clustering(x: np.array, 
                           max_clusters: int=10,
                           max_iterations: int=1000, 
                           n_samples: int=169, 
                           lucky_number: int=LUCKY_NUMBER,
                           verbose: bool=False):
    scores = {}
    for k in range(2, max_clusters-1):
        if k < 2:
            continue
        try:
            kmeans = KMeans(n_clusters=k, max_iter=max_iterations, random_state=lucky_number).fit(x)
        except:
            continue
        # scores[k] = kmeans.inertia_
        try:
            scores[k] = silhouette_score(x, kmeans.labels_, metric='euclidean', sample_size=n_samples)
        except ValueError:
            continue

    best_k = max(scores, key=scores.get)
    if verbose:
        print(f"Best k is {best_k}")
    return best_k

def reorder_cluster(cluster_field_name: str, 
                    target_field_name: str, 
                    df: pd.DataFrame, 
                    ascending: bool=True):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name, ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df, df_new[[cluster_field_name, 'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name], axis=1)
    df_final = df_final.rename(columns={"index": cluster_field_name})
    df_final[cluster_field_name] = df_final[cluster_field_name] + 1
    return df_final

def cdf(x):
    """
    Cumulative Density Function (with epsilon)
    """
    x = np.sort(x)
    u, c = np.unique(x, return_counts=True)
    n = len(x)
    y = (np.cumsum(c)-0.5) / n
    
    def interpolate_(x_):
        y_interp = np.interp(x_, u, y, left=0.0, right=1.0)
        return y_interp
    
    return interpolate_

def cumulative_kl(x, y, fraction: float=0.5): 
    """
    Cumulative Method to calculate Kullback–Leibler divergence
    """
    dx = np.diff(np.sort(np.unique(x)))
    dy = np.diff(np.sort(np.unique(y)))
    ex = np.min(dx)
    ey = np.min(dy)
    e = np.min([ex, ey]) * fraction
    n = len(x)
    P = cdf(x)
    Q = cdf(y)
    
    divergence = (1./n) * np.sum(np.log((P(x)-P(x-e)) / (Q(x)-Q(x-e)+1e-11)))
    return np.abs(divergence)

def preprocess_for_classifier(df: pd.DataFrame, target_name: str,
                              id_cols: list=[], train_size: float=0.69):
    # Split to train and validation
    dset, X, Y = dict(), dict(), dict()

    # If the minimum number of groups for any class less than 2
    try:
        dset['train'], dset['test'] = train_test_split(df, train_size=train_size, stratify=df[target_name])
        for ds_name, ds in dset.items():
            Y[ds_name] = ds[target_name]    
            X[ds_name] = ds.copy()
            X[ds_name].drop(columns=id_cols+[target_name], errors='ignore', inplace=True)
    except:
        X['train'] = df.copy()
        X['train'].drop(columns=id_cols+[target_name], errors='ignore', inplace=True)
        X['test'] = X['train'].copy()

        Y['train'] = df[target_name]
        Y['test'] = Y['train'].copy()
    
    # Compute class weights for target
    target_weights = Y['train']
    target_classes = target_weights.unique()
    class_weights = list(
        class_weight.compute_class_weight('balanced', target_classes, target_weights)
    )
    target_weights = target_weights.map({clss_i+1: clss_w for clss_i, clss_w in enumerate(class_weights)})
 
    return X, Y, target_weights

def visualize_results(classifier, X, Y):
    results = classifier.evals_result()

    epochs = len(results['validation_0']['mlogloss'])
    x_axis = range(0, epochs)

    viz_df = pd.DataFrame(classifier.feature_importances_, 
                          index=X['train'].columns, 
                          columns=['feature_importance'])
    viz_df.sort_values(by=['feature_importance'], inplace=True)
    # viz_df[viz_df.feature_importance>0.011].plot(kind='barh', alpha=0.75)

    print('\n\nAccuracy of XGB classifier on training: {:.2f}'
                .format(classifier.score(X['train'], Y['train'])))
    y_pred = classifier.predict(X['train'])
    print(classification_report(Y['train'], y_pred))

    print('\n\nAccuracy of XGB classifier on testing: {:.2f}'
                .format(classifier.score(X['test'], Y['test'])))
    y_pred = classifier.predict(X['test'])
    print(classification_report(Y['test'], y_pred))

def filter_opposite_features(df: pd.DataFrame, verbose: bool=False):
    features_before = list(df.columns)
    features_after = deepcopy(features_before)
    feature_id = 0
    while feature_id < len(features_after):
        feature_1 = features_after[feature_id]
        if verbose:
            print(f"Checking {feature_1}")
        feature_id += 1

        is_separable = False
        for op in [' = ', ' - ', ' + ', ' > ', ' < ']:
            if op in feature_1:
                is_separable = True
                break

        if not is_separable:
            continue

        obj_a, obj_b = feature_1.split(op)
        feature_2 = obj_b + op + obj_a
        if feature_2 in features_after:
            if verbose:
                print(f"Remove {feature_2} because of oppositing {feature_1}")
            features_after.remove(feature_2)

    features_removed = list(set(features_before).difference(set(features_after)))
    if verbose:
        print(f"\n\n\nFeatures removed:\n\t", features_removed)
    return features_removed

def feature_clustering(df: pd.DataFrame, 
                       feature_name: str,
                       cluster_name: str=None,
                       ascending: bool=True, 
                       reorder_by_feature: str=None,
                       best_k: int=None) -> pd.DataFrame:
    # Preprocess
    cluster_df = df[
        ['GuestID', feature_name] if not reorder_by_feature else \
        ['GuestID', feature_name, reorder_by_feature]
    ]
    x = cluster_df[feature_name].values.reshape(-1, 1)
    if not cluster_name:
        cluster_name = feature_name + '_Cluster'

    # Segmentation
    if not best_k:
        best_k = find_best_k_clustering(x)
    model = KMeans(n_clusters=best_k, random_state=LUCKY_NUMBER)
    model.fit(x)

    # Predict
    cluster_df[cluster_name] = model.predict(x)

    # Sorting
    if not reorder_by_feature:
        reorder_by_feature = feature_name
    cluster_df = reorder_cluster(cluster_name, reorder_by_feature, cluster_df, ascending)
    df = df.merge(cluster_df[['GuestID', cluster_name]], how='outer', on='GuestID')

    return df, best_k, model



In [ ]:
class Age(Variable):
    _default_pandas_dtype = int
    _name = 'Age'
    
    
class Price(Variable):
    _default_pandas_dtype = float
    _name = 'Price'


class Weight(Variable):
    _default_pandas_dtype = float
    _name = 'Weight'


class Size(Variable):
    _default_pandas_dtype = str
    _name = 'Size'
    
    
class MeasurementUnit(Variable):
    _default_pandas_dtype = str
    _name = 'MeasurementUnit'


class Currency(Variable):
    _default_pandas_dtype = str
    _name = 'Currency'

class People(Variable):
    _default_pandas_dtype = int
    _name = 'People'

CUSTOM_VARIABLES = [
    Currency, MeasurementUnit, Size, Weight, Price, Age, People
  ]

class TimeDelta(TransformPrimitive):
    
    name = "scalar_subtract_numeric_feature"
    input_types = [Datetime, Datetime]
    return_type = Numeric
    compatibility = [Library.PANDAS, Library.DASK, Library.KOALAS]

    def __init__(self):
        self.description_template = "the result of {} minus {}"

    def get_function(self):
        def time_delta_between(x_vals, y_vals):
            return (x_vals - y_vals) / pd.Timedelta(days=1)
        return time_delta_between

    def generate_name(self, base_feature_names):
        return "%s - %s" % (base_feature_names[0], base_feature_names[1])

class Quarter(TransformPrimitive):
    """
    Determines the quarter value of a datetime.
    """
    name = "quarter"
    input_types = [Datetime]
    return_type = Ordinal
    compatibility = [Library.PANDAS, Library.DASK, Library.KOALAS]
    description_template = "the quarter of {}"

    def get_function(self):
        def quarterize(vals):
            m_vals = vals.dt.month
            q_vals = m_vals.copy()
            q_vals.loc[m_vals <= 3] = 1 # 1st quarter of the year
            q_vals.loc[m_vals > 3] = 2 
            q_vals.loc[m_vals > 6] = 3 
            q_vals.loc[m_vals > 9] = 4 
            return q_vals
        return quarterize

class FeatureSelector(BaseEstimator):

    def __init__(self, problem_type: int="regression", 
                       n_runs: int=5,
                       corr_threshold: float=0.9,
                       retain_features: list=[],
                       n_workers: int=1,
                       verbose: bool=True):
        """
        Multi-step cross-validated feature selection
        
        Inputs:
            - problem_type: str, either "regression" or "classification" (default: "regression")
            - n_runs: number of times to perform feature selection with a random fraction of data points (int; default: 5)
            - corr_threshold: threshold to deliminate correlated features
            - retain_features: list of features that must be retained
            - n_jobs: how many jobs to run when selecting the features in parallel (int; default: 1)
            - n_workers: the number of workers in parallel
            - verbose: verbosity level (boolean; default: False)
        
        Attributes:
            - important_features_: list of important features (to select via pandas.DataFrame columns)
            - original_features_: list of original features of X when calling fit
            - return_df_: whether to return a pandas.DataFrame; if False, return a numpy.array
        """
        self.retain_features = retain_features
        self.corr_threshold = corr_threshold
        self.problem_type = problem_type
        self.n_workers = n_workers
        self.verbose = verbose
        self.n_runs = n_runs

    def fit(self, X: pd.DataFrame or np.array, 
                  y: pd.Series or np.array):
        # Verify data types
        if isinstance(X, pd.DataFrame):
            self.return_df_ = True
            self.original_features_ = list(X.columns)
        else:
            self.return_df_ = False
            self.original_features_ = [f"x_{i}" for i in range(X.shape[1])]
            X = pd.DataFrame(X, columns=self.original_features_)
        if not isinstance(y, pd.Series):
            y = pd.Series(y, name='target')
        
        # Perform multi-step feature selection
        self.important_features_ = feature_screening(
            X=X, y=y, 
            n_runs=self.n_runs, 
            verbose=self.verbose, 
            n_workers=self.n_workers, 
            problem_type=self.problem_type,
            corr_threshold=self.corr_threshold, 
            retain_features=self.retain_features
        )
        return self                      

    def transform(self, X: pd.DataFrame or np.array):
        # Validate attributes
        check_is_fitted(self, ["important_features_"])
        if len(self.important_features_) == 0:
            if self.verbose:
                print("WARNING: No important features found; returning data unchanged.")
            return X

        # Validate data types
        if isinstance(X, pd.DataFrame):
            features = list(X.columns)
        else:
            features = [f"x_{i}" for i in range(X.shape[1])]
        X = check_array(X, force_all_finite="allow-nan")
        if sorted(self.original_features_) != sorted(features):
            raise ValueError("Features are different from calling `fit`")

        # Get selected features only
        X = pd.DataFrame(X, columns=features)
        X_selected = X[self.important_features_]
        if self.return_df_:
            return X_selected
        return X_selected.values

    def fit_transform(self, X: pd.DataFrame or np.array, 
                            y: pd.Series or np.array):
        self.fit(X, y)
        X_transformed = self.transform(X)
        return X_transformed

# Define functions to duplicate
numeric_functions = [
    AddNumeric(),
    SubtractNumeric(),
    # MultiplyNumeric(),
    # DivideNumeric(), DivideByFeature(value=1),
    # ModuloNumeric(), ModuloByFeature(value=1),
    GreaterThan(), GreaterThanScalar(value=0), 
    LessThan(), LessThanScalar(value=0), 
    Equal(), EqualScalar(value=0), 
  ]


# Duplicate Numeric functions for customized Variables
def duplicate_functions(functions: list or tuple):
    duplicated_functions = []
    duplicated_variables = [Age, Price, Weight]
    for dup_func in functions:
        for var_type in duplicated_variables:
            # print(f"\nDuplicating function {dup_func.name} for variable {var_type._name}")

            # Define nearly-duplicated primitive
            dup_func.name = dup_func.name.replace('numeric', var_type._name.lower())
            input_types = dup_func.input_types
            if isinstance(input_types[0], (list, tuple)):
                n_inputs = len(input_types[0])
                dup_func.compatibility = [Library.PANDAS]
                dup_func.input_types = [[var_type] * n_inputs]
            else:
                n_inputs = len(input_types)
                dup_func.input_types = [var_type] * n_inputs
            
            # Append to list
            duplicated_functions.append(dup_func)
    return duplicated_functions


# Define default primitives
default_aggregation_primitives = [
    "sum", "max", "min", "mode", 
    "mean", "std", "skew",
    "count", "percent_true", "num_unique",
  ]

default_transformation_primitives = [
    # Datetime
    "month", Quarter, "weekday", "is_weekend", TimeDelta,

    # LatLong
    "haversine", 

    # NaturalLanguage
    # "num_words", "num_characters", 

    # Age
    # GroupOf
  ] + numeric_functions + duplicate_functions(numeric_functions)

#fill na
class DataImputer:
    
    def __init__(df: pd.DataFrame, 
                 feature_types: dict, 
                 imputation_methods: dict={
                     'categorical': ['constant', 'most_frequent', 'random_prob', 'knn', 'mice', 'em', 'deep_learning'],
                     'numerical': ['mode', 'mean', 'median', 'random_prob', 'knn', 'mice', 'em'],
                     'datetime': ['current_date', '1st_date']
                 }):
        self.df = df
        self.feature_types = {
            feature: feature_type.lower() for feature, feature_type in feature_types.items()
        }
        self.imputation_methods = imputation_methods
        self.imputed_features = dict()
        
    def fill_na(self):
        for feature_name in self.df.columns:
            print(f'\nImputing {feature_name} ...')
            if feature_name not in self.feature_types.keys():
                print(f'\tType of {feature_name} is not declared!')
                continue
            elif self.feature_types[feature_name] not in self.imputation_methods.keys():
                print(f'\tType {self.feature_types[feature_name]} is not supported!')
                continue
            feature = self.df[feature_name]
            
    def fill_na_by_constant(self, feature, const_type: str):
        """
        Constant value could be 1 among [0, mean, median, mode]
        Pros:
            . easy and fast
        Cons:
            . not factor the correlations between features 
            . give poor results on encoded categorical features (with MEAN, MEDIAN or MODE)
            . not very accurate
            . not account for the uncertainty in the imputations
        """
        pass

    def fill_na_by_most_frequent(self, feature):
        """
        Pros:
            . works well with categorical features
        Cons:
            . not factor the correlations between features 
            . can introduce bias in the data
        """
        pass

    def fill_na_by_random_prob(self, feature):
        pass

    def fill_na_by_kNN(self, feature):
        """
        This function creates a basic MEAN impute, then uses the resulting list to construct a KDTree. 
        Then, it uses the resulting KDTree to compute nearest neighbours (NN). 
        After it finds the k-NNs, it takes the weighted average of them.
        Pros:
            . much more accurate than the mean, median or most frequent imputation methods
        Cons:
            . computationally expensive
            . quite sensitive to outliers
        """
        pass

    def fill_na_by_MICE(self, feature):
        """
        Multivariate Imputation by Chained Equations
        """
        pass

    def fill_na_by_EM(self, feature):
        pass

    def fill_na_by_deep_learning(self, feature):
        """
        Pros:
            . quite accurate compared to other methods
            . works very well with categorical and non-numerical features
        Cons:
            . single-column imputation
            . quite slow with large datasets
        """
        pass

    def fill_na_by_date(self, feature, date_type: str):
        pass

    

In [ ]:
DEFAULT_TOP_N = 20
def auto_feature_engineering(entity_set: ft.EntitySet,
                             table_name: str,
                             max_depth: int=2,
                             max_features: int=-1,
                             seed_features: list=None,
                             encoding_categorical: bool=True,
                             verbose: bool=False) -> pd.DataFrame:

    # Deep Feature Synthesis 
    #   (Categorical features are not processed, on default)
    feature_matrix, feature_definitions = ft.dfs(
        entityset=entity_set,
        target_entity=table_name,
        agg_primitives=default_aggregation_primitives,
        trans_primitives=default_transformation_primitives,
        max_depth=max_depth,
        max_features=max_features,
        verbose=verbose,
        return_variable_types=[Numeric, Discrete, Ordinal, Categorical, Boolean]+CUSTOM_VARIABLES
    )

    # 1-hot encoding for Categorical features, including generated ones
    if encoding_categorical:
        features_before = set(feature_matrix.columns.values.tolist())
        feature_matrix, feature_definitions = encode_categorical_features(feature_matrix, feature_definitions, include_unknown=False, verbose=verbose)
        if verbose:
            features_after = set(feature_matrix.columns.values.tolist())
            features_encoded = list(features_before.difference(features_after))
            print("\n\nFeatures being encoded:\n\t", "\n\t".join(features_encoded))

    return feature_matrix.astype(float), feature_definitions


def encode_categorical_features(feature_matrix: pd.DataFrame, features, 
                                top_n=DEFAULT_TOP_N, include_unknown=True,
                                to_encode=None, inplace=False, 
                                drop_first=False, verbose=False):
    """
    Encode categorical features
        Args:
            feature_matrix (pd.DataFrame): Dataframe of features.
            features (list[PrimitiveBase]): Feature definitions in feature_matrix.
            top_n (int or dict[string -> int]): Number of top values to include.
                If dict[string -> int] is used, key is feature name and value is
                the number of top values to include for that feature.
                If a feature's name is not in dictionary, a default value of 10 is used.
            include_unknown (pd.DataFrame): Add feature encoding an unknown class.
                defaults to True
            to_encode (list[str]): List of feature names to encode.
                features not in this list are unencoded in the output matrix
                defaults to encode all necessary features.
            inplace (bool): Encode feature_matrix in place. Defaults to False.
            drop_first (bool): Whether to get k-1 dummies out of k categorical
                    levels by removing the first level.
                    defaults to False
            verbose (str): Print progress info.
        Returns:
            (pd.Dataframe, list) : encoded feature_matrix, encoded features
    """
    if not isinstance(feature_matrix, pd.DataFrame):
        raise TypeError("feature_matrix must be a Pandas DataFrame")

    X = feature_matrix if inplace else feature_matrix.copy()

    old_feature_names = set()
    for feature in features:
        for fname in feature.get_feature_names():
            assert fname in X.columns, (f"Feature {fname} not found in feature matrix")
            old_feature_names.add(fname)

    pass_through = [col for col in X.columns if col not in old_feature_names]

    iterator = make_tqdm_iterator(iterable=features,
                                  total=len(features),
                                  desc="Encoding pass 1",
                                  unit="feature") if verbose else features
    new_feature_list = []
    new_columns = []
    encoded_columns = set()

    for f in iterator:
        # TODO: features with multiple columns are not encoded by this method,
        # which can cause an "encoded" matrix with non-numeric vlaues
        is_categorical = issubclass(f.variable_type, Categorical)
        if (f.number_output_features > 1 or not is_categorical):
            if f.number_output_features > 1:
                print(f"[WARNING] Feature {f} has multiple columns and will not be encoded. This may result in a matrix with non-numeric values.")
            new_feature_list.append(f)
            new_columns.extend(f.get_feature_names())
            continue

        if to_encode is not None and f.get_name() not in to_encode:
            new_feature_list.append(f)
            new_columns.extend(f.get_feature_names())
            continue

        val_counts = X[f.get_name()].value_counts().to_frame()
        index_name = val_counts.index.name
        if index_name is None:
            if 'index' in val_counts.columns:
                index_name = 'level_0'
            else:
                index_name = 'index'
        val_counts.reset_index(inplace=True)
        val_counts = val_counts.sort_values([f.get_name(), index_name],
                                            ascending=False)
        val_counts.set_index(index_name, inplace=True)
        select_n = top_n
        if isinstance(top_n, dict):
            select_n = top_n.get(f.get_name(), DEFAULT_TOP_N)
        if drop_first:
            select_n = min(len(val_counts), top_n)
            select_n = max(select_n - 1, 1)
        unique = val_counts.head(select_n).index.tolist()
        for label in unique:
            add = f == label
            add_name = add.get_name()
            new_feature_list.append(add)
            new_columns.append(add_name)
            encoded_columns.add(add_name)
            X[add_name] = (X[f.get_name()] == label)

        if include_unknown:
            unknown = f.isin(unique).NOT().rename(f.get_name() + " is unknown")
            unknown_name = unknown.get_name()
            new_feature_list.append(unknown)
            new_columns.append(unknown_name)
            encoded_columns.add(unknown_name)
            X[unknown_name] = (~X[f.get_name()].isin(unique))

        X.drop(f.get_name(), axis=1, inplace=True)

    new_columns.extend(pass_through)
    new_X = X[new_columns]
    iterator = new_X.columns
    if verbose:
        iterator = make_tqdm_iterator(iterable=new_X.columns,
                                      total=len(new_X.columns),
                                      desc="Encoding pass 2",
                                      unit="feature")
    for c in iterator:
        if c in encoded_columns:
            try:
                new_X[c] = pd.to_numeric(new_X[c], errors='raise')
            except (TypeError, ValueError):
                pass

    return new_X, new_feature_list

def filter_out_minor_categoricals(df: pd.DataFrame, 
                                  column_names: list or tuple,
                                  top_k: int=None,
                                  min_percent: float=0.01, 
                                  verbose: bool=False):
    for col in column_names:
        if verbose:
            print(f"\n\nFiltering {col} ...")
        if col not in df.columns:
            if verbose:
                print(f"\tCannot find {col} in your dataframe!")
            continue
            
        # Check top values
        df[col] = df[col].astype(str).str.lower()
        all_values = df[col].value_counts(normalize=True)
        if top_k:
            top_values = list(all_values.index)[:top_k]
        else:
            top_values = list(all_values.loc[all_values>=min_percent].index)
            
        if verbose:
            print(f"\tAll values")
            print(all_values)
            print(f"\tTop values:", top_values)
            
        # Filter-out minority
        def filter_minority(val):
            if not val or val not in top_values:
                val = 'others'
            return val
        
        df[col] = df[col].apply(filter_minority)
        if verbose:
            print(f"\tFilter values")
            print(df[col].value_counts(normalize=True))
            
    return df

# **CODE DETECT THE CUSTOMERS**

In [ ]:
START_DATE = pd.to_datetime('2000-01-01', format='%Y-%m-%d')
END_DATE = pd.to_datetime(datetime.date(2021, 1, 1),format='%Y-%m-%d') 
# R_value = InactiveDays = Datetime(1/1/2021) - guest_df.DepartureDate.max()
# that's why we use 1/1/2021 is end date

detect new cus and returning cus

In [ ]:
def guest_type_detector(folder_path: str,folder_out_path:str):
  # Load data
  DF = dict()
  DF['guest_ltv'] = pd.read_csv(str(folder_path) + "bookings.csv")

  # Preprocess
  time_cols = [col for col in DF['guest_ltv'].columns if 'date' in col.lower()]
  DF['guest_ltv'][time_cols] = DF['guest_ltv'][time_cols].apply(pd.to_datetime, errors='ignore')
  
  # --create one matrix for copy and use  DF['guest_ltv'] because we don't want to have a problem on  DF['guest_ltv']
  visitors_df = DF['guest_ltv'].copy()

  # --Count the value of ArrivalDate on guest id. so we can know who is the fist and return
  #How can I determine who are returning / first-time customers?
  #--> Guest Type Detection: Using bookings list, you can group-by GuestID and then, 
  #count number of ArrivalDate to determine type of a guest: first-time (only 1 arrival date), or returning (many arrival dates).
  visits_counter = visitors_df[['GuestID', 'ArrivalDate']].groupby(by=['GuestID']).agg({'ArrivalDate': 'nunique'})
  visits_counter.reset_index(inplace=True)
  visits_counter.rename(columns={'ArrivalDate': 'NumberOfArrivals'}, inplace=True) #rename ready for merge 
  # --this time we have values of NumberOfArrivals, we merge to the database for detect
  DF['guest_ltv'] = DF['guest_ltv'].merge(visits_counter, how='outer', on='GuestID', sort=False)

  #ready for RFM 
  rooms_counter = visitors_df[['GuestID', 'ArrivalDate']].groupby(by=['GuestID', 'ArrivalDate']).agg({'ArrivalDate': 'count'})
  rooms_counter.rename(columns={'ArrivalDate': 'NumberOfRooms'}, inplace=True)
  rooms_counter.reset_index(inplace=True)
  DF['guest_ltv'] = DF['guest_ltv'].merge(rooms_counter, how='outer', on=['GuestID', 'ArrivalDate'], sort=False)

  # Build DataFrame for 1st Reservations
  DF['1st_reservation'] = DF['guest_ltv'].groupby(by=['GuestID']).first()
  DF['1st_reservation'].reset_index(inplace=True)
  DF['1st_reservation']['is1stVisit'] = DF['1st_reservation'].NumberOfArrivals == 1
  DF['1st_reservation'].drop(columns=['NumberOfArrivals'],inplace=True) # every first-time cus have NumberOfArrivals=1 so we do not need this
  DF['1st_reservation'].to_csv(folder_out_path + "1st_reservation.csv", index=False)

  # Build DataFrame for Returning Guests
  DF['frequent_guests'] = DF['guest_ltv'][DF['guest_ltv'].NumberOfArrivals > 1]
  DF['frequent_guests'].to_csv(folder_out_path + "returning_guests.csv", index=False)


In [ ]:
# this def for check data 
def data_impute(df: pd.DataFrame) -> pd.DataFrame:
  df = filter_out_minor_categoricals(df=df,column_names=['Channel', 'Country'])

  # Drop missing-valued samples
  df.dropna(subset=['TotalPayment', 'ArrivalDate', 'DepartureDate'], inplace=True)

  # Normalize
  df.Status = df.Status.str.upper()

  return df

# def for Recency
def generate_features_recency(guest_data: pd.DataFrame, END_DATE):
    most_recent_date = guest_data.DepartureDate.max()
    inactive_days = int((END_DATE-most_recent_date) / pd.Timedelta(days=1))
    active_months = list(guest_data.ArrivalDate.dt.month) + \
                    list(guest_data.DepartureDate.dt.month)

    most_active_month = max(set(active_months), key=active_months.count)
    most_active_quarter = month_to_quarter(most_active_month)
    least_active_month = min(set(active_months), key=active_months.count)
    least_active_quarter = month_to_quarter(least_active_month)
    
    return inactive_days, most_active_month, least_active_month, most_active_quarter, least_active_quarter

# def for Frequency
def generate_features_frequency(guest_data: pd.DataFrame, customer_lifetime,
                                date_1st_booking, END_DATE, MAM_data):
    n_orders = len(guest_data.groupby(by=['ArrivalDate']))
    average_orders = int(n_orders/customer_lifetime) if customer_lifetime>1 else n_orders
    n_orders_in_MAM = len(MAM_data)
    average_orders_in_MAM = int(n_orders_in_MAM/customer_lifetime) if customer_lifetime>1 else n_orders_in_MAM
    
    return n_orders, average_orders, average_orders_in_MAM

# def for monetary
def generate_features_monetary(guest_data: pd.DataFrame, customer_lifetime, MAM_data):
    total_revenue = guest_data.TotalPayment.sum()
    average_revenue = total_revenue/customer_lifetime if customer_lifetime>1 else total_revenue
    revenue_in_MAM = MAM_data.TotalPayment.sum()
    average_revenue_in_MAM = revenue_in_MAM/int(customer_lifetime) if customer_lifetime>1 else revenue_in_MAM
    
    return total_revenue, average_revenue, average_revenue_in_MAM


def generate_features_nonRFM(guest_data: pd.DataFrame, n_orders, total_revenue):
    nights_in_house = int(np.mean(guest_data.Nights))
    arrdates = guest_data.ArrivalDate.sort_values()
    days_between_orders = (arrdates.values[1:] - arrdates.values[:-1]) / pd.Timedelta(days=1)
    days_between_orders = np.mean(days_between_orders) if len(days_between_orders)>0 \
                            else (START_DATE-END_DATE) / pd.Timedelta(days=1)
    days_between_orders = int(days_between_orders)

    n_rooms = len(guest_data)
    average_rooms = int(n_rooms/n_orders) 
    average_revenue_per_order = total_revenue / n_orders
    average_revenue_per_room = total_revenue / n_rooms

    return days_between_orders, nights_in_house, n_rooms, \
                average_rooms, average_revenue_per_order, average_revenue_per_room


def feature_engineering_manual_for_LTV(df: pd.DataFrame) -> pd.DataFrame:
    global START_DATE, END_DATE

    features = [
        # Recency features
        'InactiveDays', 'MostActiveMonth', 'LeastActiveMonth', 'MostActiveQuarter', 'LeastActiveQuarter',
        
        # Frequency features
        'NumberOfOrders', 'AverageOrdersPerYear', 'AverageOrdersInMostActiveMonth',
        
        # Monetary features
        'TotalRevenue', 'AverageRevenuePerYear', 'AverageRevenueInMostActiveMonth',
        
        # Non-RFM features
        'AverageDaysBetweenOrders', 'AverageNightsInHouse', 'NumberOfRooms', 
        'AverageRoomsPerOrder', 'AverageRevenuePerOrder', 'AverageRevenuePerRoom',
        
        # Personal features
        'Lifetime', 'Country',
    ]
    guests_df = pd.DataFrame(columns=features)

    for guest_id, guest_data in print_progress(df.groupby(by=['GuestID'])):
    
        # Filter by time window
        # date_1st_booking = guest_data.CreatedDate.min()
        date_1st_booking = guest_data.ArrivalDate.min()
        customer_lifetime = (END_DATE-date_1st_booking).days / 365.25
        
        # Generate features for RECENCY
        inactive_days, most_active_month, least_active_month, \
                    most_active_quarter, least_active_quarter = generate_features_recency(guest_data, END_DATE)
        
        # Filter Most Active Month
        MAM_data = guest_data.loc[
            # (guest_data.CreatedDate.dt.month == most_active_month) |
            (guest_data.ArrivalDate.dt.month == most_active_month) |
            (guest_data.DepartureDate.dt.month == most_active_month)
        ]
        
        # Generate features for FREQUENCY
        n_orders, average_orders, average_orders_in_MAM = generate_features_frequency(
            guest_data, customer_lifetime, date_1st_booking, END_DATE, MAM_data
        )

        # Generate features for MONETARY
        total_revenue, average_revenue, average_revenue_in_MAM = generate_features_monetary(
            guest_data, customer_lifetime, MAM_data
        )
        
        # Generate features for LTV classification
        days_between_orders, nights_in_house, n_rooms, \
        average_rooms, average_revenue_per_order, average_revenue_per_room = generate_features_nonRFM(
            guest_data, n_orders, total_revenue
        )
        
        # Feed generated features into DataFrame
        guests_df.loc[guest_id] = [
            inactive_days, most_active_month, least_active_month, most_active_quarter, least_active_quarter,
            n_orders, average_orders, average_orders_in_MAM,
            total_revenue, average_revenue, average_revenue_in_MAM,
            days_between_orders, nights_in_house, n_rooms, average_rooms, average_revenue_per_order, average_revenue_per_room,
            customer_lifetime, guest_data.Country.unique()[0]
        ]

    guests_df['GuestID'] = list(guests_df.index)
    guests_df.reset_index(drop=True, inplace=True)

    for col in guests_df.columns:
        if col == 'Country':
            continue
        guests_df[col] = guests_df[col].astype(float)

    return guests_df

#def for LTV_returning_guests
def feature_engineering_auto_for_LTV(df: pd.DataFrame) -> pd.DataFrame:
    df_name = 'returning_guests'
    EntitySet = ft.EntitySet(id=df_name)
    EntitySet = EntitySet.entity_from_dataframe(
        entity_id=df_name,
        dataframe=df,
        make_index=False, 
        index='GuestID',
        variable_types={
            'GuestID': Id,
            'Country': Categorical,
            'Lifetime': Age,
            
            'TotalRevenue': Price,
            'AverageRevenuePerYear': Price,
            'AverageRevenueInMostActiveMonth': Price,
            
            'InactiveDays': Numeric,
            'NumberOfRooms': Numeric,
            'NumberOfOrders': Numeric,
            'AverageNightsInHouse': Numeric, 
            'AverageDaysBetweenOrders': Numeric, 
            'AverageRevenuePerOrder': Numeric,
            'AverageRevenuePerRoom': Numeric,
            'AverageRoomsPerOrder': Numeric,
            'AverageOrdersPerYear': Numeric,
            'AverageOrdersInMostActiveMonth': Numeric,
            
            'MostActiveMonth': Categorical,
            'LeastActiveMonth': Categorical,
            'MostActiveQuarter': Categorical,
            'LeastActiveQuarter': Categorical,
        }
    )
    
    feature_matrix, feature_definitions = auto_feature_engineering(
        entity_set=EntitySet, table_name=df_name, verbose=True
    )
    
    return feature_matrix

#def for 1stBooking
def feature_engineering_auto_for_1stBooking(df: pd.DataFrame) -> pd.DataFrame:
  df_name = '1st_reservation'
  EntitySet = ft.EntitySet(id=df_name)
  EntitySet = EntitySet.entity_from_dataframe(
        entity_id=df_name,
        dataframe=df,
        make_index=False, 
        index='GuestID',
        variable_types={
            'GuestID': Id,
            
            'Adults': People,
            'Children': People,
            'TotalPayment': Price,
            'Nights': Numeric,
            'RoomPrice': Numeric,
            'NumberOfRooms': Numeric,
            
            'Channel': Categorical,
            'Status': Categorical,
            'RoomGroupID': Categorical,
            
            # 'CreatedDate': Datetime,
            'ArrivalDate': Datetime,
            'DepartureDate': Datetime,
        }
    )
    
  feature_matrix, feature_definitions = auto_feature_engineering(
        entity_set=EntitySet, table_name=df_name, verbose=True
    )
    
  return feature_matrix

# this after for all 
# pipeline for load data -> progressing data(impute-> Feature)
def feature_engineering_pipeline(folder_path: str,folder_out_path:str, current_year: int = 0):
  global END_DATE

  if current_year != 0:
        input_date = str(int(current_year)+1) + "-01-01"
        END_DATE = pd.to_datetime(input_date, format='%Y-%m-%d')
  else:
        input_date = str(int(datetime.now().year)+1) + "-01-01"
        END_DATE = pd.to_datetime(input_date, format='%Y-%m-%d')

  # Load data
  DF = dict()
  df_names = ['1st_reservation', 'returning_guests']
  for df_name in df_names:
        DF[df_name] = load_csv(folder_out_path+f"{df_name}.csv")
        for col in DF[df_name].columns:
            if 'date' in col.lower():
                DF[df_name][col] = pd.to_datetime(DF[df_name][col])

  # Preprocess for raw features
  for df_name in df_names:   
        # Data Imputation
        DF[df_name] = data_impute(DF[df_name])

        # Create feature `Nights`
        DF[df_name]['Nights'] = (DF[df_name].DepartureDate - DF[df_name].ArrivalDate) / pd.Timedelta(days=1)
        DF[df_name]['Nights'] = DF[df_name]['Nights'].astype(int)

        # Save
        DF[df_name].to_csv(folder_out_path+f"{df_name}_fillna.csv", index=False)

  # Feature Engineering
  DF['returning_guests'] = feature_engineering_manual_for_LTV(DF['returning_guests'])
  DF['returning_guests'] = feature_engineering_auto_for_LTV(DF['returning_guests'])
  DF['returning_guests'].to_csv(folder_out_path+"returning_guests_processed.csv")

  dsets = DF['1st_reservation']['is1stVisit']
  dsets.index = DF['1st_reservation'].GuestID
  Xs = DF['1st_reservation'].drop(columns=['is1stVisit'], axis=1)
  Xs = feature_engineering_auto_for_1stBooking(Xs)
  DF['1st_reservation'] = pd.concat([Xs, dsets.to_frame()], axis=1)
  DF['1st_reservation'].to_csv(folder_out_path+"1st_reservation_processed.csv")

# **MODELLING**

In [ ]:
def boxcox(x, lmbda=None, alpha=None):
    
    x = np.asarray(x)
    if x.ndim != 1:
        raise ValueError("Data must be 1-dimensional.")

    if x.size == 0:
        return x

    if np.all(x == x[0]):
        raise ValueError("Data must not be constant.")

    if any(x <= 0):
        raise ValueError("Data must be positive.")

    if lmbda is not None:  # single transformation
        return special.boxcox(x, lmbda)

    # If lmbda=None, find the lmbda that maximizes the log-likelihood function.
    lmax = boxcox_normmax(x, method='mle')
    y = boxcox(x, lmax)

    if alpha is None:
        return y, lmax
    else:
        # Find confidence interval
        interval = _boxcox_conf_interval(x, lmax, alpha)
        return y, lmax, interval


In [ ]:
# def for LTV
def split_ltv(folder_path: str,folder_out_path:str):
  
  DF = pd.read_csv(folder_out_path+"returning_guests_processed.csv")

  MIN_LIFETIME = DF.Lifetime.describe().loc['50%']
  # --with k=12 month we cannot detect anything beacause of min of lifetime is 1.23 it's mean >12 mounth 
  # --split data 
  df = dict() 
  df['longlife']= DF[DF.Lifetime >= MIN_LIFETIME]
  df['shortlife']= DF[DF.Lifetime < MIN_LIFETIME]

  
  fig, ax = plt.subplots(1, 3, figsize=(12,4))
  sns.distplot(df['longlife']['InactiveDays'], ax = ax[0]) #R
  sns.distplot(df['longlife']['NumberOfOrders'], ax = ax[1]) #f
  sns.distplot(df['longlife']['TotalRevenue'], ax = ax[2]) #M
  plt.show()

  data_t=pd.DataFrame()
  data_t['GuestID']=df['longlife']['GuestID']
  data_t['InactiveDays_tran'] = scipy.stats.boxcox(df['longlife']['InactiveDays'])[0]
  data_t['NumberOfOrders_tran'] = scipy.stats.boxcox(df['longlife']['NumberOfOrders'])[0]
  data_t['TotalRevenue_tran'] = scipy.stats.boxcox(df['longlife']['TotalRevenue'])[0]

  scaler = StandardScaler()
  scaler.fit(data_t[['InactiveDays_tran','NumberOfOrders_tran','TotalRevenue_tran']])

  data_t[['InactiveDays_tran','NumberOfOrders_tran','TotalRevenue_tran']]= scaler.transform(data_t[['InactiveDays_tran','NumberOfOrders_tran','TotalRevenue_tran']])
  # -- this steps, i'll normalize the data for make three of RFM can be normal distribution
  # -- of couse this need a vision of person for have a good normallize(boxcox, sqrt, log) 
  # -- but I hope when i go to team I can develop more on this automatically and give more accurate results
  df['longlife']=df['longlife'].merge(data_t,how='outer', on='GuestID')
  df['longlife'].drop(columns=["InactiveDays","NumberOfOrders","TotalRevenue"],inplace=True)
  df['longlife'].rename(columns={"InactiveDays_tran":"InactiveDays","NumberOfOrders_tran":"NumberOfOrders","TotalRevenue_tran":"TotalRevenue"},inplace=True)
                                      
  print(df['longlife'].head())
  CLUSTEROR = dict()
  K_CLUSTERS = dict()
  WEIGHTS = {'Recency': 1, 'Frequency': 3, 'Monetary': 5}
  for feature, cluster, ascending in zip(['InactiveDays', 'NumberOfOrders', 'TotalRevenue'],
                                           ['Recency', 'Frequency', 'Monetary'], 
                                           [False, True, True]):
    df['longlife'], best_k, model = feature_clustering(df['longlife'], feature, cluster+'_Cluster', ascending)
    CLUSTEROR[cluster] = model
    K_CLUSTERS[cluster] = best_k

  df['longlife']['RFM_Score'] = WEIGHTS['Recency'] / K_CLUSTERS['Recency'] * df['longlife']['Recency_Cluster'] \
                               + WEIGHTS['Frequency'] / K_CLUSTERS['Frequency'] * df['longlife']['Frequency_Cluster'] \
                                + WEIGHTS['Monetary'] / K_CLUSTERS['Monetary'] * df['longlife']['Monetary_Cluster']
  
  df['longlife']['Estimated_LTV'] = (df['longlife'].TotalRevenue / df['longlife'].NumberOfOrders) ** (1 / (1+df['longlife'].InactiveDays/365.25))
  
  df['longlife'], _, model = feature_clustering(df['longlife'], 
                                                   feature_name='RFM_Score', 
                                                   cluster_name='LTV_Cluster', 
                                                   ascending=True, 
                                                   reorder_by_feature='Estimated_LTV', 
                                                   best_k=3)
  
  df['longlife']['LTV_Class'] = df['longlife']['LTV_Cluster'].map(
        {1: 'CLV_Low_Prob', 2: 'CLV_Mid_Prob', 3: 'CLV_High_Prob'}
    )
  df['longlife'] = pd.concat([df['longlife'], 
                                 pd.get_dummies(df['longlife'].LTV_Class)], axis=1)
  df['longlife'][
        ['GuestID', 'LTV_Class', 'CLV_Low_Prob', 'CLV_Mid_Prob', 'CLV_High_Prob']
    ].to_csv(folder_out_path+'LTV_class_of_long_time_guests.csv', index=False)

    # Model Classification
  df['longlife'].drop(columns=['Estimated_LTV', 'RFM_Score', 
                                  'InactiveDays', 'NumberOfOrders', 'TotalRevenue', 
                                  'Recency_Cluster', 'Frequency_Cluster', 'Monetary_Cluster',
                                  'LTV_Class', 'CLV_Low_Prob', 'CLV_Mid_Prob', 'CLV_High_Prob'],
                         errors='ignore', 
                         inplace=True)
  
  X, Y, target_weights = preprocess_for_classifier(df['longlife'], 
                                                  target_name='LTV_Cluster', 
                                                  id_cols=['GuestID', 'Lifetime'])
  
  classifier = XGBClassifier(max_depth=5, 
                               learning_rate=0.1, 
                               objective='multi:softprob', 
                               n_jobs=-1)
  classifier.fit(X['train'], Y['train'], 
                   sample_weight=target_weights.values, 
                   eval_metric=["mlogloss"], 
                   eval_set=[(X['train'], Y['train']), (X['test'], Y['test'])], 
                   early_stopping_rounds=7
                   )

  visualize_results(classifier, X, Y)

  print(df['longlife'].head())
  
  X['infer'] = df['shortlife'].copy()
  unused_features = []
  for col in X['infer'].columns:
        if col not in list(X['train'].columns):
          unused_features.append(col)

  X['infer'].drop(columns=unused_features, inplace=True)
  df['shortlife'][['CLV_Low_Prob', 'CLV_Mid_Prob', 'CLV_High_Prob']] = classifier.predict_proba(X['infer'])
  df['shortlife'].sort_values(by=['GuestID'], inplace=True)
  df['shortlife']['LTV_Cluster'] = df['shortlife'][['CLV_Low_Prob', 'CLV_Mid_Prob', 'CLV_High_Prob']].apply(np.argmax, axis=1)
  df['shortlife']['LTV_Cluster'] += 1
  df['shortlife']['LTV_Class'] = df['shortlife']['LTV_Cluster'].map(
        {1: 'CLV_Low_Prob', 2: 'CLV_Mid_Prob', 3: 'CLV_High_Prob'}
    )
  df['shortlife'][
        ['GuestID', 'LTV_Class', 'CLV_Low_Prob', 'CLV_Mid_Prob', 'CLV_High_Prob']
    ].to_csv(folder_out_path+'LTV_class_of_short_time_guests.csv', index=False)


In [ ]:
guest_type_detector('../input/hcg-test-data-services-dept/','./')


In [ ]:
feature_engineering_pipeline('../input/hcg-test-data-services-dept/','./',2021)

In [ ]:
split_ltv('../input/hcg-test-data-services-dept/','./')

we can see that with my normal distribution the result have the higher accuracy on training and testing data.
